In [1]:
import os
import sys


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# TODO: https://github.com/noamgat/lm-format-enforcer

In [2]:
if "notebooks" in os.getcwd():
    os.chdir("..")
    # %cd ".."

In [3]:
import torch
from datasets import load_dataset
from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

import train

/home/ubuntu/miniforge3/envs/shopgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_path = "./results/2024-04-10_12-41-01/checkpoint-10/"
data_config = "./data/shopgpt/no_prompt_mistral.yaml"

use_gpu = True
load_in_4_bit = True

In [5]:
model_kwargs = {}

if load_in_4_bit:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    model_kwargs = {**model_kwargs, "quantization_config": quantization_config}

# model = outlines.models.transformers(
#     model_path, device="cuda" if use_gpu else "cpu", model_kwargs=model_kwargs if load_in_4_bit else {}
# )

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_path,
    device_map="cuda" if use_gpu else "cpu",
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
    # use_cache=not script_args.gradient_checkpointing,
    **model_kwargs,
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:45<00:00, 26.28s/it]


In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    # padding_side="left",
    # add_eos_token=True,
    add_bos_token=True,
)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
data_config = OmegaConf.load(data_config)

# json_grammar = outlines.grammars.json
# generator = outlines.generate.cfg(model, json_grammar)

In [8]:
formatting_func = train.formatting_func(data_config.prompt)


def generate_prompt(prompt):
    prompts = formatting_func(prompt)
    # strip prompts at ```json
    prompts = [prompt.split("```json")[0] for prompt in prompts]
    # ```json
    return {"text": prompts}


dataset = load_dataset(
    data_config.dataset.type,
    data_files={"eval": data_config.dataset.eval},
)

eval_dataset = dataset["eval"].map(generate_prompt, batched=True, batch_size=8)

In [9]:
eval_dataset[0]["text"]

"<|im_start|>system You're an e-commerce product expert and understand JSON.<|im_end|> <|im_start|>user {'Produkttyp': 'Schlauchbox', 'Schlauchlänge': '15 m', 'Einsatzbereich': 'Garten', 'Farbe': 'weiß', 'Ausstattung': ['Schlauchverbinder', 'Anschlussschlauch', 'Schlauch', 'Wasserstop']} {'Produkttyp': 'Schlauchbox', 'Schlauchlänge': '20 m', 'Einsatzbereich': 'Garten', 'Farbe': 'weiß', 'Ausstattung': ['Schlauchverbinder', 'Anschlussschlauch', 'Schlauch', 'Wasserstop']} {'Produkttyp': 'Schlauchbox', 'Schlauchlänge': '20 m', 'Einsatzbereich': 'Garten', 'Ausstattung': ['Schlauchverbinder', 'Anschlussschlauch', 'Schlauch', 'Wasserstop']} <|im_end|> <|im_start|>assistant "

In [10]:
model.eval()

inputs = tokenizer(
    eval_dataset[0]["text"],
    return_tensors="pt",
)

if use_gpu:
    inputs = inputs.to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        # do_sample=True,
        # temperature=1,
        max_new_tokens=1024,
        pad_token_id=tokenizer.unk_token_id,
    )

    print(
        tokenizer.batch_decode(
            outputs.detach().cpu().numpy(),
            skip_special_tokens=True,
        )
    )

['<|im_start|> system You\'re an e-commerce product expert and understand JSON.  <|im_start|> user {\'Produkttyp\': \'Schlauchbox\', \'Schlauchlänge\': \'15 m\', \'Einsatzbereich\': \'Garten\', \'Farbe\': \'weiß\', \'Ausstattung\': [\'Schlauchverbinder\', \'Anschlussschlauch\', \'Schlauch\', \'Wasserstop\']} {\'Produkttyp\': \'Schlauchbox\', \'Schlauchlänge\': \'20 m\', \'Einsatzbereich\': \'Garten\', \'Farbe\': \'weiß\', \'Ausstattung\': [\'Schlauchverbinder\', \'Anschlussschlauch\', \'Schlauch\', \'Wasserstop\']} {\'Produkttyp\': \'Schlauchbox\', \'Schlauchlänge\': \'20 m\', \'Einsatzbereich\': \'Garten\', \'Ausstattung\': [\'Schlauchverbinder\', \'Anschlussschlauch\', \'Schlauch\', \'Wasserstop\']}   <|im_start|> assistant 1. {\'Produkttyp\': \'Schlauchbox\', \'Schlauchlänge\': \'15 m\', \'Einsatzbereich\': \'Garten\', \'Farbe\': \'weiß\', \'Ausstattung\': [\'Schlauchverbinder\', \'Anschlussschlauch\', \'Schlauch\', \'Wasserstop\']}\n `{\n  "Produkttyp": "Schlauchbox",\n  "Schla

In [11]:
# sequence = generator(eval_dataset[0]["text"])
# print(sequence)

In [12]:
eval_dataset[0]["text"]

"<|im_start|>system You're an e-commerce product expert and understand JSON.<|im_end|> <|im_start|>user {'Produkttyp': 'Schlauchbox', 'Schlauchlänge': '15 m', 'Einsatzbereich': 'Garten', 'Farbe': 'weiß', 'Ausstattung': ['Schlauchverbinder', 'Anschlussschlauch', 'Schlauch', 'Wasserstop']} {'Produkttyp': 'Schlauchbox', 'Schlauchlänge': '20 m', 'Einsatzbereich': 'Garten', 'Farbe': 'weiß', 'Ausstattung': ['Schlauchverbinder', 'Anschlussschlauch', 'Schlauch', 'Wasserstop']} {'Produkttyp': 'Schlauchbox', 'Schlauchlänge': '20 m', 'Einsatzbereich': 'Garten', 'Ausstattung': ['Schlauchverbinder', 'Anschlussschlauch', 'Schlauch', 'Wasserstop']} <|im_end|> <|im_start|>assistant "